# 基于Bert的短文本相似性模型
BERT是一个NLP任务的里程碑式模型，它的发布势必会带来一个NLP的新时代。<p>
Goole开源这个模型，并提供预训练好的模型，使得我们可以将其预训练的模型通过迁移学习的方式应用于各种NLP任务<p>
本文就将使用BERT与Goole开源的预训练模型来做短文本相似性的判定。<p>

#### 基础配置Config

In [1]:
class FLAGS:
    data_dir = 'data'
    bert_config_file = './GooleBertBaseModel/bert_config.json  '
    vocab_file='./GooleBertBaseModel/vocab.txt'
    output_dir = './sim_output/'
    init_checkpoint = './GooleBertBaseModel/bert_model.ckpt'
    
    max_seq_length = 128
    max_predictions_per_seq = 20
    do_train = True
    do_eval = True
    do_predict = False
    do_lower_case = True
    train_batch_size = 32
    eval_batch_size = 8
    predict_batch_size = 8
    learning_rate = 5e-5
    num_train_epochs = 5
    warmup_proportion = 0.1
    dopredict = True
    num_train_steps = 100000
    num_warmup_steps = 10000
    save_checkpoints_steps = 1000
    iterations_per_loop = 1000
    max_eval_steps = 100
    # money is all you need  
    use_tpu = False
    tpu_name = None
    tpu_zone = None
    gcp_project = None
    master = None
    num_tpu_cores = 8

#### 引入基础库

In [2]:
import codecs
import collections
from random import shuffle
import csv
import os
import modeling
import optimization
import tokenization
import tensorflow as tf

D:\Program Files\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### 读入数据

定义2个数据格式化的类，`输入数据格式化` `特征格式化`

In [3]:
class InputExample(object):
  """A single training/test example for simple sequence classification."""

  def __init__(self, guid, text_a, text_b=None, label=None):
    """Constructs a InputExample.

    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
    self.guid = guid
    self.text_a = text_a
    self.text_b = text_b
    self.label = label


class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self, input_ids, input_mask, segment_ids, label_id):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_id = label_id

读取

In [4]:
file_path = os.path.join(config.data_dir, 'sim_data.csv')
with codecs.open(file_path, 'r', encoding="utf-8") as f:
    reader = f.readlines()

预览

In [5]:
reader[:5]

['1\t为何我无法申请开通花呗信用卡收款\t支付宝开通信用卡花呗收款不符合条件怎么回事\t1\r\n',
 '2\t花呗分期付款会影响使用吗\t花呗分期有什么影响吗\t0\r\n',
 '3\t为什么我花呗没有临时额度\t花呗没有临时额度怎么可以负\t0\r\n',
 '4\t能不能开花呗老兄\t花呗逾期了还能开通\t0\r\n',
 '5\t我的怎么开通花呗收钱\t这个花呗是个什么啥？我没开通 我怎么有账单\t0\r\n']

In [6]:
def get_examples(data):    
    examples = []
    for index, line in enumerate(data):
        guid = 'train-%d' % index
        split_line = line.strip().split("\t")
        text_a = tokenization.convert_to_unicode(split_line[1])
        text_b = tokenization.convert_to_unicode(split_line[2])
        label = split_line[3]
        examples.append(InputExample(guid=guid, text_a=text_a,
                                     text_b=text_b, label=label))
    return examples

##### 打乱数据集

In [7]:
shuffle(reader)

预览

In [8]:
reader[:5]

['31089\t身份证过期影响借呗额度吗\t身份证过期了蚂蚁借呗用不了吗\t0\r\n',
 '20705\t蚂蚁借呗恢复额度大概需要多长时间\t借呗分期后什么时候恢复额度\t1\r\n',
 '1604\t花呗怎么让我还款，不知道还的是什么款\t我之前没有用过花呗，现在我的花呗是附属的，让我还款\t1\r\n',
 '8358\t为什么我在借呗借钱还不到帐\t我刚借呗借的钱什么时候到卡里\t0\r\n',
 '6894\t借呗还款短信提示\t借呗到账成功会有信息提示吗\t0\r\n']

##### 分割数据集
`80% train`|
`20% test`|
`20% dev`

In [9]:
split_point = int(0.8*len(reader))
train_examples,test_examples = reader[:split_point],reader[split_point:]
train_examples,test_examples,eval_examples = get_examples(train_examples),get_examples(test_examples),get_examples(test_examples)

In [10]:
train_examples[1].guid,train_examples[1].text_a,train_examples[1].text_b,train_examples[1].label

('train-1', '蚂蚁借呗恢复额度大概需要多长时间', '借呗分期后什么时候恢复额度', '1')

定义训练的类别 `0`与`1`表示`不相似`与`相似`

In [11]:
label_list = ["0", "1"]

In [12]:
tokenizer = tokenization.FullTokenizer(
      vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

In [13]:
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.contrib.tpu.RunConfig(
  cluster=None,
  master=FLAGS.master,
  model_dir=FLAGS.output_dir,
  save_checkpoints_steps=FLAGS.save_checkpoints_steps,
  tpu_config=tf.contrib.tpu.TPUConfig(
      iterations_per_loop=FLAGS.iterations_per_loop,
      num_shards=FLAGS.num_tpu_cores,
      per_host_input_for_training=is_per_host))

In [14]:
num_train_steps = int(len(train_examples) / FLAGS.train_batch_size * FLAGS.num_train_epochs)
num_warmup_steps = int(num_train_steps * FLAGS.warmup_proportion)

#### 载入BERT Goole训练好的模型的配置参数
注意 `max_seq_length`要小于谷歌开源的模型的`max_position_embeddings`。</p>
谷歌开源的模型的`max_position_embeddings`在论文中指出为512

In [15]:
bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)

In [16]:
bert_config.max_position_embeddings

512

In [17]:
tf.gfile.MakeDirs(FLAGS.output_dir)

In [18]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
  """Creates a classification model."""
  model = modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids,
      use_one_hot_embeddings=use_one_hot_embeddings)

  # In the demo, we are doing a simple classification task on the entire
  # segment.
  #
  # If you want to use the token-level output, use model.get_sequence_output()
  # instead.
  output_layer = model.get_pooled_output()

  hidden_size = output_layer.shape[-1].value

  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    if is_training:
      # I.e., 0.1 dropout
      output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    probabilities = tf.nn.softmax(logits, axis=-1)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, per_example_loss, logits, probabilities) = create_model(
        bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
        num_labels, use_one_hot_embeddings)

    tvars = tf.trainable_variables()
    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint:
      (assignment_map, initialized_variable_names
      ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:

        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:

      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op,
          scaffold_fn=scaffold_fn)
    elif mode == tf.estimator.ModeKeys.EVAL:

      def metric_fn(per_example_loss, label_ids, logits):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        accuracy = tf.metrics.accuracy(label_ids, predictions)
        loss = tf.metrics.mean(per_example_loss)
        return {
            "eval_accuracy": accuracy,
            "eval_loss": loss,
        }

      eval_metrics = (metric_fn, [per_example_loss, label_ids, logits])
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metrics=eval_metrics,
          scaffold_fn=scaffold_fn)
    else:
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode, predictions=probabilities, scaffold_fn=scaffold_fn)
    return output_spec

  return model_fn

In [19]:
model_fn = model_fn_builder(
                      bert_config=bert_config,
                      num_labels=len(label_list),
                      init_checkpoint=FLAGS.init_checkpoint,
                      learning_rate=FLAGS.learning_rate,
                      num_train_steps=num_train_steps,
                      num_warmup_steps=num_warmup_steps,
                      use_tpu=FLAGS.use_tpu,
                      use_one_hot_embeddings=FLAGS.use_tpu)

虽然是调用TPU的接口，但是如果没有配置TPU参数的话，会回调到GPU/CPU的

In [20]:
estimator = tf.contrib.tpu.TPUEstimator(
                  use_tpu=FLAGS.use_tpu,
                  model_fn=model_fn,
                  config=run_config,
                  train_batch_size=FLAGS.train_batch_size,
                  eval_batch_size=FLAGS.eval_batch_size,
                  predict_batch_size=FLAGS.predict_batch_size)

INFO:tensorflow:Using config: {'_model_dir': './sim_output/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002C8392ECC18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, computation_shape=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None), '_cluster': None}
INFO:tensorflow:_TPUContext: eval_on_tpu True


In [21]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
      break
    if len(tokens_a) > len(tokens_b):
      tokens_a.pop()
    else:
      tokens_b.pop()

def convert_single_example(ex_index, example, label_list, max_seq_length,
                           tokenizer):
  """Converts a single `InputExample` into a single `InputFeatures`."""
  label_map = {}
  for (i, label) in enumerate(label_list):
    label_map[label] = i

  tokens_a = tokenizer.tokenize(example.text_a)
  tokens_b = None
  if example.text_b:
    tokens_b = tokenizer.tokenize(example.text_b)

  if tokens_b:
    # Modifies `tokens_a` and `tokens_b` in place so that the total
    # length is less than the specified length.
    # Account for [CLS], [SEP], [SEP] with "- 3"
    _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
  else:
    # Account for [CLS] and [SEP] with "- 2"
    if len(tokens_a) > max_seq_length - 2:
      tokens_a = tokens_a[0:(max_seq_length - 2)]

  # The convention in BERT is:
  # (a) For sequence pairs:
  #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
  #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
  # (b) For single sequences:
  #  tokens:   [CLS] the dog is hairy . [SEP]
  #  type_ids: 0     0   0   0  0     0 0
  #
  # Where "type_ids" are used to indicate whether this is the first
  # sequence or the second sequence. The embedding vectors for `type=0` and
  # `type=1` were learned during pre-training and are added to the wordpiece
  # embedding vector (and position vector). This is not *strictly* necessary
  # since the [SEP] token unambiguously separates the sequences, but it makes
  # it easier for the model to learn the concept of sequences.
  #
  # For classification tasks, the first vector (corresponding to [CLS]) is
  # used as as the "sentence vector". Note that this only makes sense because
  # the entire model is fine-tuned.
  tokens = []
  segment_ids = []
  tokens.append("[CLS]")
  segment_ids.append(0)
  for token in tokens_a:
    tokens.append(token)
    segment_ids.append(0)
  tokens.append("[SEP]")
  segment_ids.append(0)

  if tokens_b:
    for token in tokens_b:
      tokens.append(token)
      segment_ids.append(1)
    tokens.append("[SEP]")
    segment_ids.append(1)

  input_ids = tokenizer.convert_tokens_to_ids(tokens)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length

  label_id = label_map[example.label]
  if ex_index < 5:
    tf.logging.info("*** Example ***")
    tf.logging.info("guid: %s" % (example.guid))
    tf.logging.info("tokens: %s" % " ".join(
        [tokenization.printable_text(x) for x in tokens]))
    tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
    tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
    tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
    tf.logging.info("label: %s (id = %d)" % (example.label, label_id))

  feature = InputFeatures(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids,
      label_id=label_id)
  return feature

In [22]:
def file_based_convert_examples_to_features(
    examples, label_list, max_seq_length, tokenizer, output_file):
  """Convert a set of `InputExample`s to a TFRecord file."""

  writer = tf.python_io.TFRecordWriter(output_file)

  for (ex_index, example) in enumerate(examples):
    if ex_index % 10000 == 0:
      tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

    feature = convert_single_example(ex_index, example, label_list,
                                     max_seq_length, tokenizer)

    def create_int_feature(values):
      f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
      return f

    features = collections.OrderedDict()
    features["input_ids"] = create_int_feature(feature.input_ids)
    features["input_mask"] = create_int_feature(feature.input_mask)
    features["segment_ids"] = create_int_feature(feature.segment_ids)
    features["label_ids"] = create_int_feature([feature.label_id])

    tf_example = tf.train.Example(features=tf.train.Features(feature=features))
    writer.write(tf_example.SerializeToString())

In [23]:
def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  name_to_features = {
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "label_ids": tf.FixedLenFeature([], tf.int64),
  }

  def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
      t = example[name]
      if t.dtype == tf.int64:
        t = tf.to_int32(t)
      example[name] = t

    return example

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.
    d = tf.data.TFRecordDataset(input_file)
    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            drop_remainder=drop_remainder))

    return d

  return input_fn

In [ ]:
if FLAGS.do_train:
    train_file = os.path.join(FLAGS.output_dir, "train.tf_record")
    file_based_convert_examples_to_features(
                                    train_examples, label_list, FLAGS.max_seq_length, tokenizer, train_file)
    tf.logging.info("***** Running training *****")
    tf.logging.info("  Num examples = %d", len(train_examples))
    tf.logging.info("  Batch size = %d", FLAGS.train_batch_size)
    tf.logging.info("  Num steps = %d", num_train_steps)
    train_input_fn = file_based_input_fn_builder(
                                    input_file=train_file,
                                    seq_length=FLAGS.max_seq_length,
                                    is_training=True,
                                    drop_remainder=True)
    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

INFO:tensorflow:Writing example 0 of 50504
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train-0
INFO:tensorflow:tokens: [CLS] 身 份 证 过 期 影 响 借 呗 额 度 吗 [SEP] 身 份 证 过 期 了 蚂 蚁 借 呗 用 不 了 吗 [SEP]
INFO:tensorflow:input_ids: 101 6716 819 6395 6814 3309 2512 1510 955 1446 7583 2428 1408 102 6716 819 6395 6814 3309 749 6010 6009 955 1446 4500 679 749 1408 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

INFO:tensorflow:Initialize variable bert/encoder/layer_0/attention/self/key/kernel:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_0/attention/self/key/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_0/attention/self/query/bias:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_0/attention/self/query/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_0/attention/self/query/kernel:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_0/attention/self/query/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_0/attention/self/value/bias:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_0/attention/self/value/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_0/attention/self/value/kernel:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_0/attention/self/value/kernel
INFO:tensorflow:Initialize variable bert/e

INFO:tensorflow:Initialize variable bert/encoder/layer_11/attention/output/LayerNorm/gamma:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_11/attention/output/LayerNorm/gamma
INFO:tensorflow:Initialize variable bert/encoder/layer_11/attention/output/dense/bias:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_11/attention/output/dense/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_11/attention/output/dense/kernel:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_11/attention/output/dense/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_11/attention/self/key/bias:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_11/attention/self/key/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_11/attention/self/key/kernel:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_11/attention/self/key/kernel
INFO:tensorflow:In

INFO:tensorflow:Initialize variable bert/encoder/layer_3/output/LayerNorm/beta:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_3/output/LayerNorm/beta
INFO:tensorflow:Initialize variable bert/encoder/layer_3/output/LayerNorm/gamma:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_3/output/LayerNorm/gamma
INFO:tensorflow:Initialize variable bert/encoder/layer_3/output/dense/bias:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_3/output/dense/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_3/output/dense/kernel:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_3/output/dense/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_4/attention/output/LayerNorm/beta:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_4/attention/output/LayerNorm/beta
INFO:tensorflow:Initialize variable bert/encoder/layer_4/attention/output/LayerN

INFO:tensorflow:Initialize variable bert/encoder/layer_6/attention/self/value/bias:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_6/attention/self/value/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_6/attention/self/value/kernel:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_6/attention/self/value/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_6/intermediate/dense/bias:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_6/intermediate/dense/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_6/intermediate/dense/kernel:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_6/intermediate/dense/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_6/output/LayerNorm/beta:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_6/output/LayerNorm/beta
INFO:tensorflow:Initialize variable bert/encoder/layer_6/o

INFO:tensorflow:Initialize variable bert/encoder/layer_9/attention/self/key/bias:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_9/attention/self/key/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_9/attention/self/key/kernel:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_9/attention/self/key/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_9/attention/self/query/bias:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_9/attention/self/query/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_9/attention/self/query/kernel:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_9/attention/self/query/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_9/attention/self/value/bias:0 from checkpoint ./GooleBertBaseModel/bert_model.ckpt with bert/encoder/layer_9/attention/self/value/bias
INFO:tensorflow:Initialize variable bert/encoder/l

INFO:tensorflow:  name = bert/encoder/layer_2/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:te

INFO:tensorflow:  name = bert/encoder/layer_7/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./sim_output/model.ckpt.


In [ ]:
if FLAGS.do_eval:
    eval_file = os.path.join(FLAGS.output_dir, "eval.tf_record")
    file_based_convert_examples_to_features(
        eval_examples, label_list, FLAGS.max_seq_length, tokenizer, eval_file)
    tf.logging.info("***** Running evaluation *****")
    tf.logging.info("  Num examples = %d", len(eval_examples))
    tf.logging.info("  Batch size = %d", FLAGS.eval_batch_size)